This notebook is part of https://github.com/AudioSceneDescriptionFormat/splines, see also https://splines.readthedocs.io/.

[back to overview](catmull-rom.ipynb)

# Derivation of Non-Uniform Catmull--Rom Splines

Multi-stage algorithm developed by
<cite data-cite="barry1988recursive">Barry and Goldman (1988)</cite>,
according to
<cite data-cite="yuksel2011parameterization">Yuksel et al. (2011)</cite>, figure 3,
which looks somewhat like this (but we shifted the indices by $+3$):

\begin{equation*}
\def\negspace{\!\!\!\!\!\!}
\begin{array}{ccccccccccccc}
&&&&&&
\boldsymbol{p}_{3,4,5,6}
&&&&&&
\\
&&&&&
\negspace \frac{t_5 - t}{t_5 - t_4} \negspace
&&
\negspace \frac{t - t_4}{t_5 - t_4} \negspace
&&&&&
\\
&&&& \boldsymbol{p}_{3,4,5} &&&& \boldsymbol{p}_{4,5,6} &&&&
\\
&&
& \negspace \frac{t_5 - t}{t_5 - t_3} \negspace && \negspace \frac{t - t_3}{t_5 - t_3} \negspace &
& \negspace \frac{t_6 - t}{t_6 - t_4} \negspace && \negspace \frac{t - t_4}{t_6 - t_4} \negspace &
&&
\\
&& \boldsymbol{p}_{3,4} &&&& \boldsymbol{p}_{4,5} &&&& \boldsymbol{p}_{5,6} &&
\\
& \negspace \frac{t_4 - t}{t_4 - t_3} \negspace && \negspace \frac{t - t_3}{t_4 - t_3} \negspace &
& \negspace \frac{t_5 - t}{t_5 - t_4} \negspace && \negspace \frac{t - t_4}{t_5 - t_4} \negspace &
& \negspace \frac{t_6 - t}{t_6 - t_5} \negspace && \negspace \frac{t - t_5}{t_6 - t_5} \negspace &
\\
\boldsymbol{x}_3 &&&& \boldsymbol{x}_4 &&&& \boldsymbol{x}_5 &&&& \boldsymbol{x}_6
\end{array}
\end{equation*}

Here we are considering the fifth spline segment
$\boldsymbol{p}_{3,4,5,6}(t)$
(represented at the tip of the triangle)
from
$\boldsymbol{x}_4$ to
$\boldsymbol{x}_5$
(to be found at the base of the triangle)
which corresponds to
the parameter range $t_4 \le t \le t_5$.
To calculate the values in this segment,
we also need to know the preceding control point $\boldsymbol{x}_3$
(at the bottom left)
and the following control point $\boldsymbol{x}_6$
(at the bottom right).
But not only their positions are relevant,
we also need the corresponding parameter values
$t_3$ and $t_6$, respectively.

## Preparations

Let's import [SymPy](https://www.sympy.org/)
and define the symbols we need:

In [ ]:
import sympy as sp
sp.init_printing()

In [ ]:
x3, x4, x5, x6 = sp.symbols('xbm3:7')

In [ ]:
t, t3, t4, t5, t6 = sp.symbols('t t3:7')

We also use some custom SymPy tools from [utility.py](utility.py):

In [ ]:
from utility import NamedExpression, NamedMatrix

The triangular figure above looks more complicated than it really is.
It's just a bunch of linear *inter*polations and *extra*polations.
Since we'll need several of those,
let's define a helper function:

In [ ]:
def lerp(xs, ts):
    """Linear interpolation.
    
    Between the two points given by *xs* in the time span given by *ts*.
    
    """
    x_begin, x_end = xs
    t_begin, t_end = ts
    return (x_begin * (t_end - t) + x_end * (t - t_begin)) / (t_end - t_begin)

Let's go through the figure above, piece by piece.

## First Stage

In the center of the bottom row,
there is a straightforward linear interpolation
from $\boldsymbol{x}_4$ to $\boldsymbol{x}_5$
within the interval from $t_4$ to $t_5$.

In [ ]:
p45 = NamedExpression('pbm_4,5', lerp((x4, x5), (t4, t5)))
p45

Obviously, this starts at:

In [ ]:
p45.evaluated_at(t, t4)

... and ends at:

In [ ]:
p45.evaluated_at(t, t5)

The bottom left of the triangle looks very similar,
with a linear interpolation
from $\boldsymbol{x}_3$ to $\boldsymbol{x}_4$
within the interval from $t_3$ to $t_4$.

In [ ]:
p34 = NamedExpression('pbm_3,4', lerp((x3, x4), (t3, t4)))
p34

However, that's not the parameter range we are interested in!
We are interested in the range from $t_4$ to $t_5$.
Therefore, this is not actually an *inter*polation between
$\boldsymbol{x}_3$ and $\boldsymbol{x}_4$,
but rather a linear *extra*polation starting at $\boldsymbol{x}_4$ ...

In [ ]:
p34.evaluated_at(t, t4)

... and ending at some extrapolated point beyond $\boldsymbol{x}_4$:

In [ ]:
p34.evaluated_at(t, t5)

Similarly, at the bottom right of the triangle
there isn't a linear *inter*polation
from $\boldsymbol{x}_5$ to $\boldsymbol{x}_6$,
but rather a linear *extra*polation that just reaches
$\boldsymbol{x}_5$ at the end of the parameter interval
(i.e. at $t=t_5$).

In [ ]:
p56 = NamedExpression('pbm_5,6', lerp((x5, x6), (t5, t6)))
p56

In [ ]:
p56.evaluated_at(t, t4)

In [ ]:
p56.evaluated_at(t, t5)

## Second Stage

The second stage of the algorithm
involves linear interpolations of the results of the previous stage.

In [ ]:
p345 = NamedExpression('pbm_3,4,5', lerp((p34.name, p45.name), (t3, t5)))
p345

In [ ]:
p456 = NamedExpression('pbm_4,5,6', lerp((p45.name, p56.name), (t4, t6)))
p456

Those interpolations are defined over a parameter range
from $t_3$ to $t_5$ and
from $t_4$ to $t_6$, respectively.
In each case, we are only interested in a sub-range,
namely from $t_4$ to $t_5$.

These are the start and end points at $t_4$ and $t_5$:

In [ ]:
p345.evaluated_at(t, t4, symbols=[p34, p45])

In [ ]:
p345.evaluated_at(t, t5, symbols=[p34, p45])

In [ ]:
p456.evaluated_at(t, t4, symbols=[p45, p56])

In [ ]:
p456.evaluated_at(t, t5, symbols=[p45, p56])

## Third Stage

The last step is quite simple:

In [ ]:
p3456 = NamedExpression('pbm_3,4,5,6', lerp((p345.name, p456.name), (t4, t5)))
p3456

This time, the interpolation interval is exactly the one we care about.

To get the final result, we just have to combine all the above expressions:

In [ ]:
p3456 = p3456.subs_symbols(p345, p456, p34, p45, p56).simplify()
p3456

We can make this marginally shorter
if we rewrite the segment durations as
$\Delta_i = t_{i+1} - t_i$:

In [ ]:
deltas = {
    t4 - t3: sp.Symbol('Delta3'),
    t5 - t4: sp.Symbol('Delta4'),
    t6 - t5: sp.Symbol('Delta5'),
    t5 - t3: sp.Symbol('Delta3') + sp.Symbol('Delta4'),
    t6 - t4: sp.Symbol('Delta4') + sp.Symbol('Delta5'),
    t6 - t3: sp.Symbol('Delta3') + sp.Symbol('Delta4') + sp.Symbol('Delta5'),
    # A few special cases that SymPy has a hard time resolving:
    t4 + t4 - t3: t4 + sp.Symbol('Delta3'),
    t6 + t6 - t3: t6 + sp.Symbol('Delta3') + sp.Symbol('Delta4') + sp.Symbol('Delta5'),
}

In [ ]:
p3456.subs(deltas)

## Basis Matrix

We already have the correct result,
but if we want to derive our *basis matrix*,
we have to re-scale this a bit.
The parameter is supposed to go from $0$ to $1$
instead of from $t_4$ to $t_5$:

In [ ]:
p3456_normalized = p3456.expr.subs(t, t * (t5 - t4) + t4)

In [ ]:
M_CR = NamedMatrix(
    r'{M_\text{CR}}',
    sp.Matrix([[c.expand().coeff(x).factor() for x in (x3, x4, x5, x6)]
               for c in p3456_normalized.as_poly(t).all_coeffs()]))
M_CR.subs(deltas)

And just to make sure that is consistent with the result
from [uniform Catmull--Rom splines](catmull-rom-uniform.ipynb),
let's set all $\Delta_i$ to $1$:

In [ ]:
uniform = [
    (t3, 3),
    (t4, 4),
    (t5, 5),
    (t6, 6),
]

In [ ]:
M_CR_uniform = NamedMatrix(
    r'{M_\text{CR,uniform}}',
    M_CR.expr.subs(uniform))

In [ ]:
M_CR_uniform.pull_out(sp.S.Half)

## Tangents

To get the tangents at $t_4$ and $t_5$,
we just have to differentiate:

In [ ]:
pd3456 = p3456.diff(t)

In [ ]:
start_tangent = pd3456.evaluated_at(t, t4)
start_tangent.subs(deltas).simplify()

In [ ]:
end_tangent = pd3456.evaluated_at(t, t5)
end_tangent.subs(deltas).simplify()

both tangents lead to the same general expression:

\begin{equation*}
\boldsymbol{\dot{x}}_i =
\frac{
(t_{i+1} - t_i)^2 (\boldsymbol{x}_i - \boldsymbol{x}_{i-1}) +
(t_i - t_{i-1})^2 (\boldsymbol{x}_{i+1} - \boldsymbol{x}_i)
}{
(t_{i+1} - t_i)(t_i - t_{i-1})(t_{i+1} - t_{i-1})
}
\end{equation*}

You might encounter another way to write the equation for $\boldsymbol{\dot{x}}_0$
(e.g. at https://stackoverflow.com/a/23980479/):

In [ ]:
(x4 - x3) / (t4 - t3) - (x5 - x3) / (t5 - t3) + (x5 - x4) / (t5 - t4)

... but this is equivalent to the equation shown above:

In [ ]:
sp.simplify(_ - start_tangent.expr)

Yet another way to skin this cat -- sometimes referred to as Bessel--Overhauser -- is to define the velocity of the left and right chords:

In [ ]:
v_left = (x4 - x3) / (t4 - t3)
v_right = (x5 - x4) / (t5 - t4)

... and then combine them in this way:

In [ ]:
((t5 - t4) * v_left + (t4 - t3) * v_right) / (t5 - t3)

Again, that's the same as we had above:

In [ ]:
sp.simplify(_ - start_tangent.expr)

## Animation

The linear interpolations (and *extra*polations) of this algorithm
can be shown graphically.

By means of the file [barry_goldman.py](barry_goldman.py),
we can generate animations of the algorithm:

In [ ]:
from barry_goldman import animation

In [ ]:
from IPython.display import HTML

In [ ]:
points = [
    (0, 0),
    (0.5, 1),
    (6, 1),
    (6.5, 0),
]

In [ ]:
times = [
    0,
    1,
    5,
    9,
]

In [ ]:
ani = animation(points, times)

In [ ]:
HTML(ani.to_jshtml(default_mode='reflect'))